<a href="https://colab.research.google.com/github/kirillspiney/projects/blob/main/statistics/w_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv'

usecols = [0, 1, 3, 4, 5, 6, 10, 22]

names = ['surgery', 'age', 'rectal_temp', 'pulse', 'resp_rate', 'extrem_temp', 'pain', 'outcome']

df = pd.read_csv(url, header=None, na_values='?', usecols=usecols, names=names)

df.head()

,surgery,age,rectal_temp,pulse,resp_rate,extrem_temp,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


In [ ]:
# создадим функцию по проверке, корректно ли заполнены данные в категориальных столбцах
def check_categorical_values(df, categorical_cols):
    for col in categorical_cols:
        valid_values = df[col].unique()
        expected_values = set()
        if col == 'surgery':
            expected_values = {1, 2}
        elif col == 'age':
            expected_values = {1, 2}
        elif col == 'extrem_temp':
            expected_values = {1, 2, 3, 4}
        elif col == 'pain':
            expected_values = {1, 2, 3, 4, 5}
        elif col == 'outcome':
            expected_values = {1, 2, 3}
        unexpected_values = set(valid_values) - expected_values
        if unexpected_values:
            print(f"Не соответствующие значения {col}: {list(unexpected_values)}")

In [ ]:
categorical_cols = ['surgery', 'age', 'extrem_temp', 'pain', 'outcome']
check_categorical_values(df, categorical_cols)

Не соответствующие значения surgery: [nan]
Не соответствующие значения age: [9]
Не соответствующие значения extrem_temp: [nan]
Не соответствующие значения pain: [nan]
Не соответствующие значения outcome: [nan]


In [ ]:
print(df['age'].unique())

# т.к. в столбце age ожидаются значения 1 и 2, а есть только значения 1 и 9, можно резонно предположить, что это ошибка ввода.

df['age'] = df['age'].replace(9, 2)

[1 9]


Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [ ]:
df.describe()

,surgery,age,rectal_temp,pulse,resp_rate,extrem_temp,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.080000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,0.271746,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [ ]:
# т. к. у нас есть как категориальные, так и количественные столбцы, применим разные способы к поиску выбросов.

# количественные
for col in ["rectal_temp", "pulse", "resp_rate"]:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    print(f"{col}: выбросы {outliers.tolist()}")

rectal_temp: выбросы [39.9, 35.4, 40.3, 39.7, 36.4, 40.3, 39.6, 36.5, 36.0, 36.1, 36.6, 40.8, 40.0, 36.5]
pulse: выбросы [164.0, 150.0, 160.0, 184.0, 150.0]
resp_rate: выбросы [84.0, 96.0, 72.0, 80.0, 80.0, 68.0, 96.0, 66.0, 68.0, 90.0, 80.0, 70.0, 88.0, 84.0, 68.0, 90.0, 70.0]


In [ ]:
# качественные
for col in ["surgery", "age", "extrem_temp", "pain", "outcome"]:
    # модальные значения
    mode = df[col].mode().tolist()
    print(f"{col}: модальные значения {mode}")
        
    # распределение по категориям
    freq = df[col].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
    print(f"{col}: распределение по категориям\n{freq}\n")
        
    # уникальные значения
    unique = df[col].unique().tolist()
    print(f"{col}: уникальные значения {unique}\n")

surgery: модальные значения [1.0]
surgery: распределение по категориям
1.0    60.2%
2.0    39.8%
Name: surgery, dtype: object

surgery: уникальные значения [2.0, 1.0, nan]

age: модальные значения [1]
age: распределение по категориям
1    92.0%
2     8.0%
Name: age, dtype: object

age: уникальные значения [1, 2]

extrem_temp: модальные значения [3.0]
extrem_temp: распределение по категориям
3.0    44.67%
1.0    31.97%
2.0     12.3%
4.0    11.07%
Name: extrem_temp, dtype: object

extrem_temp: уникальные значения [3.0, nan, 1.0, 4.0, 2.0]

pain: модальные значения [3.0]
pain: распределение по категориям
3.0    27.35%
2.0    24.08%
5.0    17.14%
4.0    15.92%
1.0    15.51%
Name: pain, dtype: object

pain: уникальные значения [5.0, 3.0, 2.0, nan, 4.0, 1.0]

outcome: модальные значения [1.0]
outcome: распределение по категориям
1.0    59.53%
2.0    25.75%
3.0    14.72%
Name: outcome, dtype: object

outcome: уникальные значения [2.0, 3.0, 1.0, nan]



Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
# пропуски
df.isna().sum()

surgery         1
age             0
rectal_temp    60
pulse          24
resp_rate      58
extrem_temp    56
pain           55
outcome         1
dtype: int64

In [ ]:
'''Столбец "surgery". В этом столбце всего один пропуск. Для заполнения пропуска можно использовать моду - наиболее часто встречающееся значение.
В данном случае это будет значение 1.'''
df["surgery"].fillna(1, inplace=True)

'''Столбец "age". В этом столбце нет пропусков.'''

'''Столбец "rectal_temp". В этом столбце 60 пропусков. Можно использовать различные методы заполнения:
Медиана: этот метод будет хорошим выбором, если распределение данных примерно нормальное, но можно использовать и другие методы. 
В данном случае медиана равна 38,2. Можно заполнить все пропуски значением 38.2.
Заполнение с помощью других переменных: можно использовать другие переменные, которые коррелируют с температурой, например, пульс или респираторный ритм, 
для заполнения пропусков в этом столбце. Однако, поскольку столбцы "pulse" и "resp_rate" также содержат пропущенные значения, 
то эти переменные нельзя использовать для заполнения пропусков в "rectal_temp".'''
df["rectal_temp"].fillna(df["rectal_temp"].median(), inplace=True)

'''Столбец "pulse". В этом столбце 24 пропуска. Можно использовать аналогичные методы заполнения:
Медиана: медианное значение для этого столбца равно 64, что можно использовать для заполнения всех пропусков в этом столбце.
Заполнение с помощью других переменных: в данном случае мы можем использовать столбец "pain" для заполнения пропусков в столбце "pulse", 
так как больные лошади с большей вероятностью будут испытывать болезненность, что может привести к увеличению пульса. Оставим медиану.'''
df["pulse"].fillna(df["pulse"].median(), inplace=True)

'''Столбец "resp_rate". В этом столбце 58 пропусков. Так как значение этого показателя может меняться в зависимости от возраста лошади, 
то мы можем использовать столбец "age" для заполнения пропусков. Можно заполнить пропуски средним значением для каждой группы возраста.'''
df["resp_rate"].fillna(df.groupby("age")["resp_rate"].transform('mean'), inplace=True)

'''Столбец "pain". Заполняем сначала его, чтобы потом использовать для "extrem_temp". В этом столбце 55 пропусков. 
Для заполнения пропусков можно использовать моду, так как этот показатель является категориальным. 
Можно заполнить все пропуски значением "3", что является наиболее часто встречающимся значением в этом столбце.'''
df["pain"].fillna(df["pain"].mode()[0], inplace=True)

'''Столбец "extrem_temp". В этом столбце 56 пропусков. Этот показатель может меняться в зависимости от состояния здоровья животного. 
Можно использовать столбец "pain" для заполнения пропусков, так как больные лошади могут испытывать экстремальную температуру. Используем моду.'''
df["extrem_temp"].fillna(df.groupby("pain")["extrem_temp"].transform(lambda x: x.mode()[0]), inplace=True)

'''Столбец "outcome". В этом столбце один пропуск. Для заполнения пропуска можно использовать моду, так как этот показатель также является категориальным.'''
df["outcome"].fillna(df["outcome"].mode()[0], inplace=True)

df.head()

,surgery,age,rectal_temp,pulse,resp_rate,extrem_temp,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,1.0,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,1.0,3.0,2.0


In [ ]:
df.isna().sum()

surgery        0
age            0
rectal_temp    0
pulse          0
resp_rate      0
extrem_temp    0
pain           0
outcome        0
dtype: int64

In [ ]:
df.to_csv('horse_data_filled.csv', index=False)